In [1]:
import os
import os.path as osp
import numpy as np
import torch, torchvision
from torch import nn
from torch.utils import model_zoo
from torch.nn.parallel import DataParallel
from torch.utils.data import DataLoader
from torch.utils.data.dataset import ConcatDataset
%load_ext autoreload
%autoreload 2

In [2]:
import hashlib
import mmcv
def gt_as_det(img_prefix, ann_dir='annotations', img_dir='images'):
    """ What we need is list of dict with:
        {'score', 'bbox', 'image_id', 'category_id'}
    """
    all_images = osp.join(img_prefix, img_dir)
    all_annos = osp.join(img_prefix, ann_dir)

    detections = []
    for img_ind, img_name in enumerate(os.listdir(all_images)):
        stem = img_name.split('.')[0]
        ann_name = stem + '.txt'
        img_full_path = osp.join(all_images, img_name)
        ann_full_path = osp.join(all_annos, ann_name)

        image_id = int(hashlib.sha256(stem.encode('utf8')).hexdigest(), 16) % (10 ** 8)

        with open(ann_full_path, 'r') as f:
            lines = f.readlines()
            lines = [v.strip('\n') for v in lines]
            lines = [v.split(',') for v in lines]
            lines = np.asarray(lines)[:, :8].astype(np.int32)

        for line in lines:
            x1, y1, w, h, sc, label, trun, occ = line
            label = int(label)
            if label == 0 or label == 11:
                # ignore ignore(0) and others(11)
                continue
            assert label > 0 and label < 11, 'Bad annotation label'
            x2, y2 = x1 + w, y1 + h
            # label is number, '0' is background
            bbox = np.asarray([x1, y1, x2, y2]).tolist()
            id = str(stem) + str(bbox)
            id = int(hashlib.sha256(id.encode('utf8')).hexdigest(), 16) % (10 ** 12)

            _det = dict(
                image_id=image_id,
                bbox=bbox,
                category_id=label,
                score=1,
            )
            detections.append(_det)

    return detections

In [3]:
detections = gt_as_det('/home/damon/DATASETS/Drone2019/VisDrone2019-DET/VisDrone2018-DET-val/')

In [4]:
save_file = '/tmp/gtdt.json'
mmcv.dump(detections, save_file)

# Now eval on coco

In [5]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [16]:
cocoGt = COCO('/home/damon/DATASETS/Drone2019/VisDrone2019-DET/VisDrone2018-DET-val/annotations_val.json')

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


In [17]:
cocoDt = cocoGt.loadRes(save_file)

Loading and preparing results...
DONE (t=0.18s)
creating index...
index created!


In [8]:
imgIds = sorted(cocoGt.getImgIds())
# imgIds = imgIds[0:100]
# imgId = imgIds[np.random.randint(100)]

In [9]:
annType = 'bbox'
cocoEval = COCOeval(cocoGt, cocoDt, annType)
cocoEval.params.imgIds = imgIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=21.99s).
Accumulating evaluation results...
DONE (t=0.51s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=10

In [11]:
cocoEval.stats

array([1.57119735e-05, 3.79253954e-05, 6.72295380e-07, 1.71017102e-04,
       9.28217822e-04, 3.90604691e-06, 1.47677956e-04, 1.75664960e-04,
       1.88720224e-04, 2.15485493e-05, 1.21083761e-04, 2.78930496e-03])